#KRW Group Project: Building Narratives from Knowledge Graphs
##Group number: P5-2
###Group members: Fina Polat, Hein Kolk, Jelle Wassenaar, Siddharth Chaubal

This is the first notebook of a series of 3.

Research goal: We are a newspaper agency and want to develop a system to create articles semi-automatically. The goal is to create a newspaper story using information from existing KGs, and to help readers better understanding the content/setting of the story (e.g. visualise a timeline to understand big events and actors in a political or historical event, summarising a movie or the life of someone, etc.).

We are going to generate a gossip story using T5 language model (LM). In order to do that we are going to:
* pre-process WebNLG Dataset - Part 1
* fine-tune T5 language model with WebNLG Dataset. - Part 2
* automatically generate stories (template + automatically generated text) - Part 3

The WebNLG data (Gardent el al., 2017) was created to promote the development (i) of RDF verbalisers and (ii) of microplanners able to handle a wide range of linguistic constructions.

T5 Language Model : Colin Raffel et al. “Exploring the limits of transfer learning with a unified
text-to-text transformer”. In: arXiv preprint arXiv:1910.10683 (2019).

The code in this notebook is adapted from https://github.com/MathewAlexander/T5_nlg

In [ ]:
### import the required libraries ###

import pandas as pd
import os

In [ ]:
import os
from google.colab import drive
MOUNTPOINT = '/content/gdrive'
DATADIR = os.path.join(MOUNTPOINT, 'My Drive', 'KRW_P5-2')
drive.mount(MOUNTPOINT)

Mounted at /content/gdrive


In [ ]:
print(DATADIR)

/content/gdrive/My Drive/KRW_P5-2


In [ ]:
import urllib.request
import zipfile
url = 'https://gitlab.com/shimorina/webnlg-dataset/-/archive/master/webnlg-dataset-master.zip?path=release_v3.0/en/train'
urllib.request.urlretrieve(url, 'web.zip')
with zipfile.ZipFile('web.zip', 'r') as zip_ref:
    zip_ref.extractall('web')
import glob
import os
import re
import xml.etree.ElementTree as ET
import pandas as pd
files = glob.glob("/content/web/webnlg-dataset-master-release_v3.0-en-train/release_v3.0/en/train/**/*.xml", recursive=True)
triple_re=re.compile('(\d)triples')
data_dct={}
for file in files:
    tree = ET.parse(file)
    root = tree.getroot()
    triples_num=int(triple_re.findall(file)[0])
    for sub_root in root:
        for ss_root in sub_root:
            strutured_master=[]
            unstructured=[]
            for entry in ss_root:
                unstructured.append(entry.text)
                strutured=[triple.text for triple in entry]
                strutured_master.extend(strutured)
            unstructured=[i for i in unstructured if i.replace('\n','').strip()!='' ]
            strutured_master=strutured_master[-triples_num:]
            strutured_master_str=(' && ').join(strutured_master)
            data_dct[strutured_master_str]=unstructured
mdata_dct={"prefix":[], "input_text":[], "target_text":[]}
for st,unst in data_dct.items():
    for i in unst:
        mdata_dct['prefix'].append('webNLG')
        mdata_dct['input_text'].append(st)
        mdata_dct['target_text'].append(i)


df=pd.DataFrame(mdata_dct)
df.to_csv('/content/gdrive/My Drive/KRW_P5-2/webNLG2020_train.csv')

In [ ]:
#Lets check the file:
train_df=pd.read_csv('/content/gdrive/My Drive/KRW_P5-2/webNLG2020_train.csv', index_col=[0])
#Let's inspect the dataset:
train_df.head

<bound method NDFrame.head of        prefix                                         input_text  \
0      webNLG  103_Colmore_Row | architect | John_Madin && 10...   
1      webNLG  103_Colmore_Row | architect | John_Madin && 10...   
2      webNLG  103_Colmore_Row | architect | John_Madin && 10...   
3      webNLG  103_Colmore_Row | architect | John_Madin && 10...   
4      webNLG  103_Colmore_Row | architect | John_Madin && 10...   
...       ...                                                ...   
35205  webNLG  Turkey | leaderTitle | President_of_Turkey && ...   
35206  webNLG  Turkey | leaderTitle | President_of_Turkey && ...   
35207  webNLG  Turkey | leaderTitle | President_of_Turkey && ...   
35208  webNLG  Turkey | leaderTitle | President_of_Turkey && ...   
35209  webNLG  Turkey | leaderTitle | President_of_Turkey && ...   

                                             target_text  
0      Architect John Madin, designed 103 Colmore Row...  
1      John Madin designed 103 Colm

End of the Part 1